
# Telecom X — Churn de Clientes (ETL + EDA)

**Autor:** Paulo Rodrigues (assistido por GPT)  
**Objetivo:** Analisar a evasão de clientes (churn) da Telecom X, seguindo boas práticas de ETL e EDA.

---



## 1. Setup
Instalação de bibliotecas (se necessário), importação e configuração básica.


In [ ]:

# !pip install -r ../requirements.txt

import os, json, re, math, textwrap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

# Pastas
BASE_DIR = Path("..").resolve()
DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
PROC_DIR = DATA_DIR / "processed"
FIG_DIR = BASE_DIR / "reports" / "figures"

for p in [DATA_DIR, RAW_DIR, PROC_DIR, FIG_DIR]:
    p.mkdir(parents=True, exist_ok=True)

RAW_URL = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"
DICT_URL = None  # opcional: informe a URL do dicionário de dados se disponível



## 2. Extração (E - Extract)
Carregando o JSON diretamente da API (GitHub Raw) e convertendo para `DataFrame`.


In [ ]:

import requests

r = requests.get(RAW_URL, timeout=60)
r.raise_for_status()
data = r.json()

with open(RAW_DIR / "telecomx_raw.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

# Normaliza em DataFrame (lida tanto com lista de dicts quanto dict com chave 'data')
if isinstance(data, dict) and "data" in data:
    df = pd.json_normalize(data["data"])
else:
    df = pd.json_normalize(data)

print("Linhas x Colunas:", df.shape)
df.head(3)



## 3. Conhecendo o Dataset
Tipos, amostras, estatísticas e (opcional) dicionário de dados.


In [ ]:

df_info = df.info()
display(df.describe(include="all", datetime_is_numeric=True).transpose())


In [ ]:

# (Opcional) Carregar dicionário de dados se você tiver a URL
if DICT_URL:
    try:
        dd = requests.get(DICT_URL, timeout=60).json()
        print("Exemplo de dicionário carregado (primeiras chaves):", list(dd)[:10])
    except Exception as e:
        print("Falha ao carregar dicionário:", e)
else:
    print("Sem dicionário de dados externo. Prosseguindo com inferência automática.")



## 4. Verificando Inconsistências
Valores ausentes, duplicados, categorias estranhas.


In [ ]:

# Duplicados
dup_count = df.duplicated().sum()
print("Duplicados:", dup_count)

# Ausentes
missing = df.isna().sum().sort_values(ascending=False)
display(missing.to_frame("missing").assign(pct=lambda x: (x["missing"]/len(df)).round(4)))



## 5. Tratamento (T - Transform)
- Padronização de nomes de colunas  
- Conversão de tipos numéricos  
- Preenchimento simples de ausentes  
- Normalização de rótulos "Sim/Não"  
- Criação da coluna `Contas_Diarias`


In [ ]:

# Funções utilitárias (espelhadas em src/etl.py)
def _clean_col(col):
    col = col.strip()
    col = re.sub(r"\s+", "_", col)
    col = col.replace("%","pct").replace("$","dolar")
    return col.lower()

work = df.rename(columns={c: _clean_col(c) for c in df.columns}).copy()

# Tentar converter colunas tipo string numéricas
for c in work.columns:
    if work[c].dtype == "object":
        try:
            work[c] = (
                work[c].astype(str)
                .str.replace(r"[R$\s]", "", regex=True)
                .str.replace(".", "", regex=False)
                .str.replace(",", ".", regex=False)
            )
            work[c] = pd.to_numeric(work[c], errors="ignore")
        except Exception:
            pass

# Relatório de ausentes (após tipagem)
missing_after = work.isna().sum().sort_values(ascending=False)
display(missing_after.to_frame("missing").assign(pct=lambda x: (x["missing"]/len(work)).round(4)))

# Preenchimento simples
for c in work.columns:
    if work[c].dtype.kind in "biufc":
        if work[c].isna().any():
            work[c] = work[c].fillna(work[c].median())
    else:
        if work[c].isna().any():
            work[c] = work[c].fillna("Desconhecido")

# Normalização Sim/Não
yes_no_map = {"Sim":1, "Não":0, "Yes":1, "No":0, "Y":1, "N":0, "S":1, "NÃO":0, "NAO":0}
for c in work.columns:
    if work[c].dtype == "object":
        uniques = set(map(str, work[c].dropna().unique()))
        if uniques & set(yes_no_map.keys()):
            work[c] = work[c].map(yes_no_map).fillna(work[c])

# Criação Contas_Diarias a partir de 'monthlycharges' se existir
if "monthlycharges" in work.columns:
    work["contas_diarias"] = work["monthlycharges"] / 30.0

work = work.drop_duplicates()
print("Shape pós-tratamento:", work.shape)

# Salvar processado
work.to_parquet(PROC_DIR / "telecomx_processed.parquet", index=False)
work.head(3)



## 6. Análise Descritiva (L - Load & Analysis)
Métricas centrais para variáveis numéricas e cardinalidade para categóricas.


In [ ]:

# Estatísticas numéricas
display(work.describe().transpose())

# Cardinalidade de categóricas
cat_cols = [c for c in work.columns if work[c].dtype == "object" or work[c].dtype.name == "category"]
card = {c: work[c].nunique(dropna=False) for c in cat_cols}
pd.DataFrame.from_dict(card, orient="index", columns=["n_unique"]).sort_values("n_unique", ascending=False)



## 7. Distribuição da Evasão (Churn)


In [ ]:

# Tenta detectar a coluna de churn automaticamente
possible = [c for c in work.columns if "churn" in c or "evasao" in c]
if not possible:
    raise ValueError("Coluna de churn não encontrada. Verifique o nome no dataset.")
CHURN_COL = possible[0]
print("Usando coluna de churn:", CHURN_COL)

counts = work[CHURN_COL].value_counts(dropna=False)
ax = counts.plot(kind="bar")
ax.set_title("Distribuição de Evasão (Churn)")
ax.set_xlabel("Churn")
ax.set_ylabel("Quantidade")
plt.tight_layout()
plt.savefig(FIG_DIR / "churn_distribution.png", dpi=150)
plt.show()

churn_rate = work[CHURN_COL].mean() if work[CHURN_COL].dtype.kind in "biufc" else (work[CHURN_COL].astype(str).str.lower().isin(["1","sim","yes","true"])).mean()
print(f"Taxa de churn (aprox.): {churn_rate:.2%}")



## 8. Churn por Variáveis Categóricas
Gráficos empilhados com proporção de churn por categoria.


In [ ]:

max_unique = 20
cat_cols = [c for c in work.columns if work[c].dtype == "object" or work[c].dtype.name == "category"]

for c in cat_cols:
    if c == CHURN_COL: 
        continue
    if work[c].nunique(dropna=False) > max_unique:
        continue
    ct = pd.crosstab(work[c], work[CHURN_COL], normalize="index")
    ax = ct.plot(kind="bar", stacked=True)
    ax.set_title(f"Churn por {c}")
    ax.set_xlabel(c)
    ax.set_ylabel("Proporção")
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"churn_by_{c}.png", dpi=150)
    plt.show()



## 9. Churn por Variáveis Numéricas
Boxplots para comparar distribuições entre churn x não-churn.


In [ ]:

num_cols = [c for c in work.columns if work[c].dtype.kind in "biufc"]
for c in num_cols:
    if c == CHURN_COL:
        continue
    ax = work.boxplot(column=c, by=CHURN_COL)
    plt.title(f"{c} por {CHURN_COL}")
    plt.suptitle("")
    plt.xlabel("Churn")
    plt.ylabel(c)
    plt.tight_layout()
    plt.savefig(FIG_DIR / f"{c}_by_{CHURN_COL}.png", dpi=150)
    plt.show()



## 10. Segmentos com Maior Churn
Top segmentos para foco em retenção (categóricas) e faixas (numéricas).


In [ ]:

def churn_rate(series):
    # tenta interpretar 1 como churn
    if series.dtype.kind in "biufc":
        return series.mean()
    s = series.astype(str).str.lower().isin(["1","sim","yes","true"])
    return s.mean()

# Garante vetor binário de churn
if work[CHURN_COL].dtype.kind in "biufc":
    churn_bin = work[CHURN_COL].astype(int)
else:
    churn_bin = work[CHURN_COL].astype(str).str.lower().isin(["1","sim","yes","true"]).astype(int)

# Categóricas
summary_rows = []
for c in cat_cols:
    if c == CHURN_COL: 
        continue
    if work[c].nunique(dropna=False) > 20:
        continue
    grp = work.groupby(c)[churn_bin.name if hasattr(churn_bin, "name") else CHURN_COL].mean()
    summary_rows.append((c, grp.sort_values(ascending=False).head(3)))

# Numéricas (binning em 5 faixas)
num_summary = []
for c in num_cols:
    if c == CHURN_COL:
        continue
    try:
        bins = pd.qcut(work[c], q=5, duplicates="drop")
        grp = work.groupby(bins)[churn_bin].mean()
        num_summary.append((c, grp.sort_values(ascending=False).head(3)))
    except Exception:
        pass

print("Top segmentos por categóricas (maior churn):")
for c, top in summary_rows[:5]:
    display(top.to_frame("churn_rate"))

print("Top faixas por numéricas (maior churn):")
for c, top in num_summary[:5]:
    display(top.to_frame("churn_rate"))



## 11. Conclusões, Insights e Recomendações
Abaixo um template que você pode adaptar após revisar os gráficos/segmentos encontrados.



**Principais achados (exemplos):**
- A taxa de churn global é de **X%**.
- Clientes com **tipo de contrato = mensal** apresentam maior churn que contratos anuais.
- **Método de pagamento = boleto** correlaciona-se com uma taxa maior de churn em relação a **cartão/crédito automático**.
- **Contas_Diarias** e **tenure** (tempo de casa) mostram que clientes com gastos mensais mais baixos e pouco tempo de contrato tendem a churnar mais.

**Recomendações (exemplos):**
- Incentivar **migração para contratos de longo prazo** (descontos/benefícios de fidelidade).
- Oferecer **upgrades de plano** ou bundles para clientes de baixo gasto mensal.
- Campanha proativa para clientes **recém-adquiridos** (primeiros 3 meses), com suporte dedicado.
- Revisar políticas de **método de pagamento**, incentivando débito automático com benefícios.
- Implementar **modelo preditivo de churn** usando as features de maior sinal para priorizar ações de retenção.
